In [ ]:
import pandas as pd

df = pd.read_csv('data/data4.tsv', delimiter = '\t')

df

In [ ]:
import numpy as np
import os

def load_contours_csv(file, size = 500):
    d = np.loadtxt(file, delimiter = ',')[:size]
    return np.pad(d, ((size - len(d), 0), (0, 0)), 'constant', constant_values = ([0, 0], [0, 0]))

points = np.array([load_contours_csv(f"data/img/{os.path.splitext(os.path.basename(i))[0]}.csv") for i in df['image'].values])

points.shape


In [ ]:
import cv2

cols = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8']

f_img = cv2.imread(df['image'][0])

w = f_img.shape[1]
h = f_img.shape[0]

dv = df[cols].astype(np.float)

for i in range(0, len(cols), 2):
    dv[cols[i]] /= w
    dv[cols[i + 1]] /= h

labels = dv.values

labels.shape

In [ ]:
labels[0]

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, GRU, Masking
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import Bidirectional

input = Input(shape = imgs.shape[1:])

x = Masking()(input)

x = Bidirectional(GRU(256))(x)
x = Dropout(0.2)(x)

x = Dense(128, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(128, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(64, activation = 'relu')(x)
x = Dropout(0.2)(x)

output = Dense(16, activation = 'sigmoid')(x)

model = Model(inputs = input, outputs = output)
          
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mae'])

model.summary()

In [ ]:
hist = model.fit(points, labels, epochs = 50, batch_size = 10)
hist

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8.0, 4.0)

plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'])

plt.subplot(1, 2, 2)
plt.plot(hist.history['mean_absolute_error'])


In [ ]:
model.save('model/rnn-1_0.h5')

In [ ]:
def scale(d):
    for i in range(0, len(d), 2):
        d[i] *= w
        d[i + 1] *= h
        
    return d

def predict(i, s = 6.0):
    plt.rcParams['figure.figsize'] = (s, s)

    img = cv2.imread(df['image'][i])
    
    r = scale(np.copy(labels[i])).astype(int)
    p = scale(model.predict(np.array([points[i]]))[0]).astype(int)
   
    print(r)
    print(p)
    
    line_size = 3
    
    r_color = (0, 255, 0)
    p_color = (255, 0, 0)

    for i in range(0, len(r), 4):
        img = cv2.line(img, (r[i], r[i + 1]), (r[i + 2], r[i + 3]), r_color, line_size)
        img = cv2.line(img, (p[i], p[i + 1]), (p[i + 2], p[i + 3]), p_color, line_size)
                       
    plt.imshow(img)
    

In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)

In [ ]:
predict(3)

In [ ]:
predict(4)